In [1]:
import pandas as pd

In [2]:
#脱ブタン塔のプロセスデータを読み込む
df = pd.read_csv('debutanizer_data.csv')

# 目的変数の測定誤差を考慮
df['y'] = df['y'].shift(5)

#yがnanとなる期間のデータを削除
df = df.dropna()

### 時間遅れ変数（ラグ特徴量）の作成

In [3]:
#説明変数と目的変数にわける
X = df.iloc[:, :-1]
y = df['y']

# 最大の遅延時間と遅延間隔（スパン）を設定
delay_max = 30
delay_span = 3
X_with_delays = X.copy()    

# Xの各列について
for col in X.columns:
    # 時間遅れ変数を追加
    for delay in range(delay_span, delay_max + 1, delay_span):
        col_name = f"{col}_delay_{delay}"
        X_with_delays[col_name] = X[col].shift(delay)

# 目標データを追加
X_with_delays['y'] = y

# 欠損値がある行を削除
X_with_delays_fil = X_with_delays.dropna(how='any')

# データを保存
X_with_delays_fil.to_csv('time_delay_df.csv')
X_with_delays_fil.to_pickle('time_delay_df.pkl')

# 入力データと目標データに分割
X = X_with_delays_fil.iloc[:, :-1]
y = X_with_delays_fil['y']

### 二乗項、交差項の作成

In [4]:
#説明変数と目的変数にわける
X = df.iloc[:, :-1]
y = df['y']

# 元の説明変数のデータセット
original_X = X.copy() 

# 二乗項を追加
X_squared = X**2
X_squared.columns = [f"{col}^2" for col in X.columns]
X = pd.concat([X, X_squared], axis=1)

# 交差項を追加
for i, col1 in enumerate(original_X.columns):
    print(i + 1, '/', original_X.shape[1])
    for col2 in original_X.columns[i+1:]:
        X[f"{col1}*{col2}"] = X[col1] * X[col2]

# データを保存
X.to_csv('squared_interaction_terms.csv')
X.to_pickle('squared_interaction_terms.pkl')

1 / 7
2 / 7
3 / 7
4 / 7
5 / 7
6 / 7
7 / 7
